In [1]:
!pip install ipython-autotime

In [2]:
%load_ext autotime

time: 0 ns (started: 2021-09-08 14:54:03 -07:00)


## Bayesian methods of hyperparameter optimization

In addition to the random search and the grid search methods for selecting optimal hyperparameters, we can use Bayesian methods of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

You will need to install the Bayesian optimization module. Running a cell with an exclamation point in the beginning of the command will run it as a shell command — please do this to install this module from our notebook in the cell below.

In [3]:
! pip install bayesian-optimization

time: 2.53 s (started: 2021-09-08 14:54:03 -07:00)


In [4]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

time: 703 ms (started: 2021-09-08 14:54:05 -07:00)


In [5]:
#proven
#conda install -c conda-forge lightgbm ############<<<<<<-------- done in terminal in same folder as this notebook


time: 0 ns (started: 2021-09-08 14:54:06 -07:00)


In [6]:
#unproven
#!yes | conda install -c conda-forge lightgbm ????

#!yes | conda uninstall lightgbm ????


time: 16 ms (started: 2021-09-08 14:54:06 -07:00)


In [7]:
import lightgbm

time: 78 ms (started: 2021-09-08 14:54:06 -07:00)


In [8]:
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool

time: 63 ms (started: 2021-09-08 14:54:06 -07:00)


In [9]:
import os
os.listdir()

['.ipynb_checkpoints',
 'Bayesian_optimization_case_study.ipynb',
 'flight_delays_test.csv.zip',
 'flight_delays_train.csv.zip']

time: 0 ns (started: 2021-09-08 14:54:06 -07:00)


## How does Bayesian optimization work?

Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

## Let's look at a simple example

The first step is to create an optimizer. It uses two items:
* function to optimize
* bounds of parameters

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. Smaller metrics are best. Hint: don't forget to use negative metric values.

Here we define our simple function we want to optimize.

In [10]:
def simple_func(a, b):
    return a + b

time: 0 ns (started: 2021-09-08 14:54:06 -07:00)


Now, we define our bounds of the parameters to optimize, within the Bayesian optimizer.

In [11]:
optimizer = BayesianOptimization(
    simple_func,
    {'a': (1, 3),
    'b': (4, 7)})

time: 0 ns (started: 2021-09-08 14:54:06 -07:00)


These are the main parameters of this function:

* **n_iter:** This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

* **init_points:** This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

Let's run an example where we use the optimizer to find the best values to maximize the target value for a and b given the inputs of 3 and 2.

In [12]:
optimizer.maximize(3,2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
|  1        |  7.557    |  2.618    |  4.939    |
|  2        |  8.693    |  2.795    |  5.898    |
|  3        |  6.669    |  1.625    |  5.044    |
|  4        |  7.342    |  2.718    |  4.625    |
|  5        |  10.0     |  3.0      |  7.0      |
time: 79 ms (started: 2021-09-08 14:54:06 -07:00)


Great, now let's print the best parameters and the associated maximized target.

In [13]:
print(optimizer.max['params']);optimizer.max['target']

{'a': 3.0, 'b': 7.0}


10.0

time: 0 ns (started: 2021-09-08 14:54:06 -07:00)


## Test it on real data using the Light GBM

The dataset we will be working with is the famous flight departures dataset. Our modeling goal will be to predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset. As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.

**<font color='teal'> You can load the zipped csv files just as you would regular csv files using Pandas read_csv. In the next cell load the train and test data into two seperate dataframes. </font>**


In [14]:
train_df = pd.read_csv('flight_delays_train.csv.zip')
test_df = pd.read_csv('flight_delays_test.csv.zip')

time: 219 ms (started: 2021-09-08 14:54:06 -07:00)


**<font color='teal'> Print the top five rows of the train dataframe and review the columns in the data. </font>**

In [15]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


time: 0 ns (started: 2021-09-08 14:54:07 -07:00)


**<font color='teal'> Use the describe function to review the numeric columns in the train dataframe. </font>**

In [16]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


time: 15 ms (started: 2021-09-08 14:54:07 -07:00)


Notice, `DepTime` is the departure time in a numeric representation in 2400 hours. 

 **<font color='teal'>The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.</font>**

In [17]:
#train_df = train_df[train_df.DepTime <= 2400].copy()
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

time: 0 ns (started: 2021-09-08 14:54:07 -07:00)


## Feature Engineering
Use these defined functions to create additional features for the model. Run the cell to add the functions to your workspace.

In [18]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period 
    return np.cos(value)

def feature_eng(df):
    df['flight'] = df['Origin']+df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['middle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int) 
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

time: 15 ms (started: 2021-09-08 14:54:07 -07:00)


Concatenate the training and testing dataframes.


In [19]:
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])
full_df = feature_eng(full_df)

time: 1.5 s (started: 2021-09-08 14:54:07 -07:00)


In [20]:
full_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Month,200000.0,6.530710,3.415164,1.0,4.000000,7.000000,9.000000,12.0
DayofMonth,200000.0,15.702585,8.792117,1.0,8.000000,16.000000,23.000000,31.0
DayOfWeek,200000.0,3.942155,1.991469,1.0,2.000000,4.000000,6.000000,7.0
Distance,200000.0,726.263635,568.955768,30.0,319.000000,575.000000,954.000000,4962.0
begin_of_month,200000.0,0.296975,0.456927,0.0,0.000000,0.000000,1.000000,1.0
middle_of_month,200000.0,0.328635,0.469718,0.0,0.000000,0.000000,1.000000,1.0
end_of_month,200000.0,0.374390,0.483966,0.0,0.000000,0.000000,1.000000,1.0
hour,200000.0,13.105345,4.792361,0.0,9.000000,13.000000,17.000000,25.0
morning,200000.0,0.319885,0.466433,0.0,0.000000,0.000000,1.000000,1.0
day,200000.0,0.431065,0.495226,0.0,0.000000,0.000000,1.000000,1.0


time: 156 ms (started: 2021-09-08 14:54:08 -07:00)


Apply the earlier defined feature engineering functions to the full dataframe.

In [21]:
for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']:
    full_df[column] = label_enc(full_df[column])

time: 313 ms (started: 2021-09-08 14:54:08 -07:00)



Split the new full dataframe into X_train and X_test. 

In [22]:
X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

time: 0 ns (started: 2021-09-08 14:54:09 -07:00)


Create a list of the categorical features.

In [23]:
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 
                        'Origin', 'Dest', 'flight', 
                        'flightUC', 'DestUC', 'OriginUC']

time: 0 ns (started: 2021-09-08 14:54:09 -07:00)


Let's build a light GBM model to test the bayesian optimizer.

### [LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

First, we define the function we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.

In [24]:
def lgb_eval(num_leaves, max_depth, lambda_l2, lambda_l1, min_child_samples, min_data_in_leaf):
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    lgtrain = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
    cv_result = lightgbm.cv(params,
                            lgtrain,
                            1000,
                            early_stopping_rounds=100,
                            stratified=True,
                            nfold=3)
    return cv_result['auc-mean'][-1]

time: 0 ns (started: 2021-09-08 14:54:09 -07:00)


Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [25]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 0.05),
                                                'lambda_l1': (0.0, 0.05),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000)
                                                })

lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
|  1        |  0.7436   |  0.02078  |  0.03743  |  9.532    |  1.991e+0 |  1.866e+0 |  3.847e+0 |
|  2        |  0.7438   |  0.01014  |  0.02885  |  35.36    |  9.025e+0 |  1.623e+0 |  1.348e+0 |
|  3        |  0.7338   |  0.04271  |  0.04885  |  55.37    |  6.083e+0 |  1.146e+0 |  1.183e+0 |
|  4        |  0.7147   |  0.01743  |  0.03946  |  57.21    |  8.782e+0 |  366.9    |  117.8    |
|  5        |  0.7337   |  0.03677  |  0.02938  |  34.29    |  512.8    |  1.14e+03 |  2.518e+0 |
|  6        |  0.7435   |  0.03887  |  0.01004  |  37.77    |  637.3    |  1.999e+0 |  111.3    |
|  7        |  0.7437   |  0.02588  |  0.03327  |  21.11    |  6.549e+0 |  1.977e+0 |  1.062e+0 |
|  8        |  0.7429   |  0.0475   |  0.004907 |  6.67     |  9.896e+0 |  1.983e+0 |  3.98e+03 |
|  9        |  0.743

 **<font color='teal'> Print the best result by using the '.max' function.</font>**

In [26]:
lgbBO.max

{'target': 0.7440270918255832,
 'params': {'lambda_l1': 0.014483930737365104,
  'lambda_l2': 0.015255565010762508,
  'max_depth': 59.008746736216224,
  'min_child_samples': 9634.948487864878,
  'min_data_in_leaf': 1988.9097803484144,
  'num_leaves': 28.12495903749087}}

time: 0 ns (started: 2021-09-08 14:58:17 -07:00)


Review the process at each step by using the '.res[0]' function.

In [27]:
lgbBO.res[0]

{'target': 0.743617296011187,
 'params': {'lambda_l1': 0.020777467154830823,
  'lambda_l2': 0.03743329811142075,
  'max_depth': 9.531726898090804,
  'min_child_samples': 1990.5689656221186,
  'min_data_in_leaf': 1865.5184004556031,
  'num_leaves': 3846.565992695298}}

time: 0 ns (started: 2021-09-08 14:58:17 -07:00)
